# Project 3

## Getting Insights into Images and their Metadata


Analysis will include:
...

In [1]:
%matplotlib inline
import scipy
import scipy.io
import numpy as np
import torch
import torchvision
from torchvision import models, transforms
import sqlite3
import matplotlib
from matplotlib import pyplot as plt
from PIL import Image

In [2]:
vgg16 = torchvision.models.vgg16(pretrained=True) # very slow command

C:\Users\saraw\miniconda3\envs\mlfds\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\saraw\miniconda3\envs\mlfds\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


funny, for me that command took maybe half a second? /sara

In [24]:
# vgg16.features is a series of modules.
# we can chain an input through the modules via the forward method.
phi = vgg16.features.forward

def preprocess_image(img):

    # commented out lines are (probably) unnecessary? I hope?
    transform = transforms.Compose([
        #transforms.Resize(256), # or 224
        #transforms.CenterCrop(224),
        transforms.ToTensor(),
        #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    batch = transform(img)
    # batch = torch.unsqueeze(batch, 0)
    return batch 

def fetch_img(img_index:int, healthy:bool=True):
    if healthy:
        return Image.open(f"Apple___healthy/image ({img_index}).JPG")
    else:
        return Image.open(f"Apple___Black_rot/image ({img_index}).JPG")

def img_features(img_index:int, healthy:bool=True):
    return phi(preprocess_image(fetch_img(img_index, healthy=healthy)))

phi_out_shape = img_features(1, healthy=True).shape
phi_out_shape


torch.Size([512, 8, 8])

actually I think centercrop and resizing to the size the training data had is relativly common but i don't know if thats neccessary here, I guess it wouldnt work otherwise? /sara <br> anyway, I will also reshape it nevertheless so its 1D and the we'll see /sara

In [25]:
def mean_features(indices, healthy=True):
    mean = None
    for x in indices:
        if mean is None:
            mean = img_features(x, healthy=healthy)
        mean += img_features(x, healthy=healthy)
    return mean

training_indices = range(1, 6) # first few images as training set. Can always be changed later.

mean_healthy = mean_features(training_indices, healthy=True)
mean_rotted = mean_features(training_indices, healthy=False)

w = mean_rotted - mean_healthy
w = torch.transpose( w / torch.linalg.norm(w), 0, 1 )

def guess(x):
    return torch.tensordot(w, phi(x))

def img_guess(img_index, healthy=True): # g(x) from the project3 pdf
    return torch.tensordot(w, img_features(img_index, healthy=True))

# see if we can correctly identify the first healthy/unhealthy leaves not in the training set
test_healthy = img_guess(11, healthy=True)
test_unhealthy = img_guess(11, healthy=False)

test_healthy, test_unhealthy

(tensor([[ 6.5840e-01,  5.8461e-01,  7.9042e-01,  1.3707e+00,  1.6708e+00,
           9.6479e-01,  7.9480e-01,  1.8551e+00],
         [ 3.0120e-01, -9.8316e-02, -7.9643e-01, -2.9018e-01,  8.7823e-01,
           7.2210e-01,  9.0211e-01,  1.1072e+00],
         [-3.3040e-02, -1.1047e+00, -2.0431e+00, -1.6285e+00, -8.1169e-01,
          -6.4916e-01, -1.3468e-03,  2.5760e-01],
         [-2.6506e-01, -1.3277e+00, -2.1925e+00, -1.7456e+00, -1.1781e+00,
          -8.9657e-01, -1.8042e-01, -7.4197e-02],
         [ 1.2060e-01,  9.2153e-02, -3.5975e-01, -3.1272e-01, -2.0766e-01,
          -2.7387e-01,  1.2102e-01,  3.2935e-01],
         [ 8.8628e-01, -5.6385e-02, -6.7700e-01, -4.5892e-01, -3.0320e-01,
          -5.9503e-01, -1.0174e-01,  8.9331e-01],
         [ 1.8325e-01, -2.1189e+00, -2.5124e+00, -1.6861e+00, -1.1622e+00,
          -1.3821e+00, -1.0510e+00,  2.6827e-01],
         [-1.6990e-01, -7.8924e-01, -5.1853e-01,  1.4417e-02, -1.2932e-01,
          -2.1154e-01, -1.7538e-01,  9.4159e-02]],